In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random, copy

In [29]:
M = np.array([[0,1,1,0,0,0],
              [1,0,0,1,1,0],
              [1,0,0,1,0,0],
              [0,1,1,0,1,0],
              [0,1,0,1,0,1],
              [0,0,0,0,1,0]])

### 1.a

- Le multi graphe G est représenté par une matrice d'adjacence.
- Pour réaliser l'opération de contraction d'une arête (i,j), on réalise les traitements suivants
sur la matrice d'adjacence représentant le graphe G:<br>
- Ajouter à la ligne et colonne du sommet i, la ligne et colonne du sommmet j <br>
- Suprimer la ligne et la colonne correspondants au sommet j (car a été fusionné avec le sommet i pour construire un nouvel ensemble.

In [30]:
def contract(A,i,j):
    """
    Realise the contraction in the graph M represented by its adjancency matrix
    """
    li = A[i,:]
    lj = A[j,:]
    ci = A[:,i]
    cj = A[:,j]

    A[i,:] = li + lj
    A[:,i] = ci + cj

    A[i,i] = 0

    A[j,:] = -1
    A[:,j] = -1
    
    return A

### 1.b

In [31]:
def create_graph(n_vertex,ty = "random"):
    shape = (n_vertex,n_vertex)
    if ty=="random":
        M = np.random.randint(0,2,(shape))
        np.fill_diagonal(M,0)
    elif ty=="complete":
        M = np.ones(shape)
        np.fill_diagonal(M,0)
    elif ty=="biparti":
        n = n_vertex//2
        A = np.concatenate((np.zeros((n,n)), np.ones((n,n))), axis=0)
        B = np.concatenate((np.ones((n,n)), np.zeros((n,n))), axis=0)
        M = np.concatenate((A,B), axis=1)
    return M.astype(int)

In [32]:
G_random = create_graph(6,ty="random")
G_complete = create_graph(6,ty="complete")
G_biparti = create_graph(6,ty="biparti")

### 1.c

le graphe G est possiblement devenu un multi-graphe avec plusieurs arêtes entre deux sommets.
Dans notre analyse théorique de la complexité de l'algorithme de Karger, nous avons supposée
que l'arêtes e de l'opération de sélection aléeatoire dans la boucle tant que est
tirée uniformement aléatoirement parmi toutes les arêtes possibles. 

Pour effectuer ce tirage aléatoire lorsque le multi-graphe G est représenté par une matrice d'adjacence, on réalise les étapes suivantes :

Etant donnée une liste somme_ligne de tailles nombre de sommets dans le graphe G, et contenant dans chaque case le nombre de sommmets par ligne dans la matrice d'adjacence :

Exemple :

Pour la matrice  $\begin{bmatrix}
0 & 1 & 1\\ 
1 & 0 & 1\\ 
1 & 0 & 0
\end{bmatrix}$ on retrouve la liste : $\begin{bmatrix}
2 & 2 & 1
\end{bmatrix}$

Donc, pour choisir aléatoirement selon une loi uniforme une arête parmi toutes les arêtes du graphe, on tire un nombre aléatoire entre 1 et le nombre d'arêtes du graphe G (qui n'est rien d'autre que la somme des éléments de la liste sommme_liste).

Soit a le numéro de l'arête qu'on a tiré précédemment, en utilisant la somme cummulée des éléments de la liste somme_ligne, on peut reperer la ligne ou se trouve cette arête. Cette étape se réalise en temps linéaire O(n) dans le pire des cas.

De même pour la colonne, on fait la somme cummulé et on retrouve le numéro de colonne. Cette étape se réalise également en temps linéaire O(n) dans le pire des cas.

A la fin de cette procédure, on retourne une l'indice de l'arête (i,j) tiré aléatoirement du graphe G en temps linéaire.


In [15]:
# Implémentation de la fonction de tirage aléatoire d'arête
# Random choice
def arrete_choice(M, somme_ligne) :
    """
    Chooses a random edge from the multi graph G
    represented by its adjacency matrix
        M : Adjacency matrix
        somme_ligne : list containing number of edges per line in M
    returns :
        M[i,j],i,j
    """
    # Pick up up a random number of edge
    nb_arrete = np.sum(somme_ligne)
    random_arrete = np.random.randint(nb_arrete)
    
    # Find in linear time the line index of that edge
    cumul = 0
    for i in range(len(somme_ligne)) :
        cumul += somme_ligne[i]
        if cumul > random_arrete :
            break
    
    # Find in linear time the column index of that edge
    j = random_arrete - (cumul - somme_ligne[i])
    cumul = 0
    for k in range(M.shape[1]) :
        cumul += M[i,k]
        if cumul > j :
            break

    return M[i,k],i,k

### 1.d

In [26]:
def Karger(M, verbose= False) :
    # Show the resulting list
    def print_fusion(fusion):
        for i in range(len(fusion)):
            if len(fusion[i])>0:
                print(fusion[i])
    
    # Init
    A = M.copy()
    fusion = [[i] for i in range(A.shape[0])]
    
    
    sum_lines = np.sum(A,axis=1)
    nb_vertex = A.shape[0]
    
    # While the number of vertex is higher than 2
    while nb_vertex > 2 :
        # An inner loop to avoid choosing an edge that has
        # a negative value in the adjacency matrix
        while True : 
            val,i,j = arrete_choice(A, sum_lines)
            if val>0:
                break
        # realises a contraction on the edge (i,j)
        A = contract(A,i,j)
        
        ## Update of the data structures
        
        # Append the merged vertex
        for e in fusion[j]:
            fusion[i].append(e)
        fusion[j] = []
        
        # Update the sum_lines list
        sum_lines[i] += sum_lines[j] - val
        sum_lines[j] = 0
        # We have merged two vertex
        nb_vertex-=1
        # Log
        if verbose:
            print("---")
            print("for debug")
            print("---")
            print(A)
            print("sommets: ",nb_vertex)
            print("somme ligne :", sum_lines)
            
    if verbose :
        print_fusion(fusion)
    
    S = []
    for i in range(len(fusion)):
        if len(fusion[i])>0:
            S.append(fusion[i])
    
    coupure = A[A>0][0]
    
    return S, coupure

# I think I found the bug
# In fact, I shouldn't be choosing the points where 
# the line/column has already been "removed" or put to -1

In [27]:
S, cut = Karger(M)
print("V :", S)
print("The cut returned by Karger Algorithm :", cut)

V : [[2, 0], [3, 1, 4, 5]]
The cut returned by Karger Algorithm : 2


### 1.e

La deuxième structure de données pour laquelle nous avons opté est la liste d'adjacence.

In [52]:
#input can be any adjacency list in a text file
data = open("testCase.txt", "r")
G = {}
for line in data:
    lst = [int(s) for s in line.split()]  # creates a list for each line of text file
    G[lst[0]] = lst[1:]  # adds the list for one vertex to the entire adj list in G
    # The code that creates an adjacency list from a text file(lines 6-10) were done with help from online resources
    
def choose_random_key(G):
    a = random.choice(list(G.keys()))  # picks a random vertex
    b = random.choice(list(G[a]))      # picks a random edge in the adjacency list
    return a, b


def karger(G):
    S = dict.fromkeys(range(len(G)), [])
    print(G)
    while len(G) > 2:
        a, b = choose_random_key(G)  # the nodes that are part of the random edge
        G[a].extend(G[b])  # copies all items Linear time

        for x in G[b]:
            G[x].remove(b)  # remove the other occurrences of b in adjacency list
            G[x].append(a)  # tell them to look at a instead
            
        while a in G[a]:
            G[a].remove(a)  # remove self loops
            
        del G[b]  # getting rid of the empty vertex from the adj list
        
        S[a].append(b)
        # Debug
        print(a,b)
        print(G)
    return S,len(G[a])  # length of either vertex is same so just picked a


def multipleKarger(iterations):
    i = 0
    count = 10000000000  # a very large number that isn't a possible mincut
    while i < iterations:
        data = copy.deepcopy(G)  # this way don't have to reread the txt file
        min_cut = karger(data)
        if min_cut < count:
            count = min_cut
        i = i + 1
    return count


print("Running Karger one time: ", karger(G))
#print("Running Karger one time: ", multipleKarger(1))
#print("Running Karger 100 times: ", multipleKarger(100))

{1: [2, 3, 4, 7], 2: [1, 3, 4], 3: [1, 2, 4], 4: [1, 2, 3, 5], 5: [4, 6, 7, 8], 6: [5, 7, 8], 7: [1, 5, 6, 8], 8: [5, 6, 7]}
6 8
{1: [2, 3, 4, 7], 2: [1, 3, 4], 3: [1, 2, 4], 4: [1, 2, 3, 5], 5: [4, 6, 7, 6], 6: [5, 7, 5, 7], 7: [1, 5, 6, 6]}
1 2
{1: [3, 4, 7, 3, 4], 3: [1, 4, 1], 4: [1, 3, 5, 1], 5: [4, 6, 7, 6], 6: [5, 7, 5, 7], 7: [1, 5, 6, 6]}
4 3
{1: [4, 7, 4, 4, 4], 4: [1, 5, 1, 1, 1], 5: [4, 6, 7, 6], 6: [5, 7, 5, 7], 7: [1, 5, 6, 6]}
1 4
{1: [7, 5], 5: [6, 7, 6, 1], 6: [5, 7, 5, 7], 7: [1, 5, 6, 6]}
1 5
{1: [7, 6, 7, 6], 6: [7, 7, 1, 1], 7: [1, 6, 6, 1]}
1 6
{1: [7, 7, 7, 7], 7: [1, 1, 1, 1]}
Running Karger one time:  ({0: [8, 2, 3, 4, 5, 6], 1: [8, 2, 3, 4, 5, 6], 2: [8, 2, 3, 4, 5, 6], 3: [8, 2, 3, 4, 5, 6], 4: [8, 2, 3, 4, 5, 6], 5: [8, 2, 3, 4, 5, 6], 6: [8, 2, 3, 4, 5, 6], 7: [8, 2, 3, 4, 5, 6]}, 4)


---

---

## Exercice 02:

### 2.b 

In [6]:
def KargerItere(M,T):
    i = 0
    min_cut = np.inf
    S = []
    
    for i in range(T):
        S1, m = Karger(M)
        if m<min_cut:
            min_cut = m
            S = S1
    
    return S, min_cut

In [7]:
KargerItere(M, T = 100)

NameError: name 'Karger' is not defined

---

## Exercice 03:

In [ ]:
def ContractionPartielle(G, t):
    